# Multistage Stochastic Transportation Problem

In [ ]:
using JuMP, SDDP, HiGHS, Gurobi, Statistics, Test

## Toy Instance: Single (Entry-Exit) Lane and Two Sources

In [ ]:
# In Julia this struct is immutable
struct HyperParamsSingle
    tau             ::Int64
    nSources        ::Int64
    entry_stock_0   ::Int64
    exit_stock_0    ::Int64
    exit_short_0    ::Int64
    entry_capacity  ::Int64
    exit_capacity   ::Int64
    flow_support    ::Vector{Int64}
    entry_store_coef::Float64
    exit_store_coef ::Float64
    exit_short_coef ::Float64
    transport_coef  ::Vector{Float64}
end

# Outer constructor with arguments
function HyperParamsSingle(; 
                     tau             ::Int64 = 4,
                     nSources        ::Int64 = 2,
                     entry_stock_0   ::Int64 = 5,
                     exit_stock_0    ::Int64 = 0,
                     exit_short_0    ::Int64 = 0,
                     entry_capacity  ::Int64 = 50,
                     exit_capacity   ::Int64 = 50,
                     flow_support    ::Vector{Int64} = collect(1:10),
                     entry_store_coef::Float64 = 2.0,
                     exit_store_coef ::Float64 = 1.0,
                     exit_short_coef ::Float64 = 3.0,
                     transport_coef  ::Vector{Float64} = [1.1, 0.7])
    HyperParamsSingle(
        tau,
        nSources,
        entry_stock_0,
        exit_stock_0,
        exit_short_0,
        entry_capacity,
        exit_capacity,
        flow_support,
        entry_store_coef,
        exit_store_coef,
        exit_short_coef,
        transport_coef)
end

In [ ]:
function transportation_t(sp::Model, stage::Int64; config::HyperParamsSingle)
    ## State variables
    @variables(sp, begin
        0 <= entry, (SDDP.State, initial_value = config.entry_stock_0)
        0 <= exitp, (SDDP.State, initial_value = config.exit_stock_0)
        0 <= exitm, (SDDP.State, initial_value = config.exit_short_0)
        0 <= move[k = 1:config.nSources]
        flow[i = 1:2]
    end)
    ## Decision variables
    ## Constraints
    # Carrier capacity
    @constraint(sp, sum(move) <= entry.in + flow[1])
    # Storage limit
    @constraint(sp, entry.in + flow[1]   <= config.entry_capacity)
    @constraint(sp, exitp.in + sum(move) <= config.exit_capacity)
    # Transition
    @constraint(sp, entry.out == entry.in + flow[1] - sum(move))
    @constraint(sp, exitp.out - exitm.out == exitp.in - exitm.in + sum(move) - flow[2])
    # Uncertain variables
    # Ξ = vec(collect(Base.product(config.flow_support, config.flow_support)))
    Ξ = vec(collect(Base.product(ntuple(_ -> config.flow_support, 2)...)))
    SDDP.parameterize(sp, Ξ) do ξ
        JuMP.fix.(flow, ξ)
    end
    ## Objective function
    @stageobjective(sp, 
        config.entry_store_coef * entry.in + 
        config.exit_store_coef  * exitp.in + 
        config.exit_short_coef  * exitm.in + 
        sum(config.transport_coef[k] * move[k] for k in 1:config.nSources)
    )
end

In [ ]:
# function transportation_t(sp::Model, stage::Int64; config::HyperParams)
#     ## State variables
#     @variables(sp, begin
#         0 <= entry[i = 1:config.nOrigins],      (SDDP.State, initial_value = config.entry_stock_0[i])
#         0 <= exitp[j = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_stock_0[j])
#         0 <= exitm[j = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_short_0[j])
#         0 <= move[k  = 1:config.nLanes]
#         inflow[ i = 1:config.nOrigins]
#         outflow[j = 1:config.nDestinations]
#     end)
#     ## Decision variables
#     ## Constraints
#     # Carrier capacity
#     @constraint(sp, [k = 1:config.nCarriers], sum(move[config.CarrierIdx[k]]) <= config.carrier_capacity[(k - 1) * config.tau + stage])
#     # Entry volume
#     @constraint(sp, [i = 1:config.nOrigins], sum(move[config.from_[i]]) <= entry[i].in + inflow[i])
#     # Storage limit
#     @constraint(sp, [j = 1:config.nDestinations], exitp[j].in + sum(move[config.to_[j]]) <= config.exit_capacity[j])
#     # Transition
#     @constraint(sp, [i = 1:config.nOrigins],      entry[i].out == entry[i].in + inflow[i] - sum(move[config.from_[i]]))
#     @constraint(sp, [j = 1:config.nDestinations], exitp[j].out - exitm[j].out == exitp[j].in - exitm[j].in + sum(move[config.to_[j]]) - outflow[j])
#     # Uncertain variables
#     # Ξ = config.flow_support
#     Ξ = config.flow_support[rand(1:length(config.flow_support), 10)]
#     # Ξ = [ rand(Poisson(1000), config.nOrigins + config.nDestinations) for _ in 1:10 ]
#     # Ξ = [[
#     #     287, 310, 286, 323, 309, 289, 297, 316, 
#     #     298, 315, 291, 309, 291, 338, 298, 289, 
#     #     305, 332, 309, 323, 337, 285, 299, 292, 
#     #     285, 296, 314, 329, 273, 317, 276, 293, 
#     #     307, 296, 304, 303, 303, 291, 288, 330, 
#     #     331, 308, 299, 297, 309, 310, 293, 312, 
#     #     301, 304, 316, 278, 275, 304, 292, 293, 
#     #     292, 284, 296, 298, 281, 317, 321, 304, 
#     #     323, 313, 312, 291, 293, 316, 305, 317, 
#     #     296, 312, 291, 293, 285, 301, 255, 284, 
#     #     287, 294, 275, 307, 319, 301, 337, 311, 
#     #     297, 344, 295, 308, 315, 303, 303, 306][(stage - 1) * 8 .+ (1:8)]]
#     SDDP.parameterize(sp, Ξ) do ξ
#         JuMP.fix.(inflow,  ξ[1:config.nOrigins])
#         JuMP.fix.(outflow, ξ[config.nOrigins .+ (1:config.nDestinations)])
#     end
#     ## Objective function
#     @stageobjective(sp, 
#         sum(config.entry_store_coef[i] * entry[i].in for i in 1:config.nOrigins)      + 
#         sum(config.exit_store_coef[j]  * exitp[j].in for j in 1:config.nDestinations) + 
#         sum(config.exit_short_coef[j]  * exitm[j].in for j in 1:config.nDestinations) + 
#         sum(sum(config.transport_coef[config.CarrierIdx[k]] .* move[config.CarrierIdx[k]]) for k in 1:config.nCarriers)
#     )
# end

In [ ]:
# Hyperparameter configuration
config = HyperParamsSingle() # Default

In [ ]:
model = SDDP.PolicyGraph(
    (sp, stage) -> transportation_t(sp, stage; config = config),
    SDDP.LinearGraph(config.tau);
    sense       = :Min,
    lower_bound = 0.0,
    optimizer   = HiGHS.Optimizer,
)

In [ ]:
SDDP.train(model;
    iteration_limit = 100,
    cut_type        = SDDP.SINGLE_CUT)
)

## Multi-Dimensional Instance

In [ ]:
using JuMP, SDDP, HiGHS, Gurobi, Statistics, Test, DataStructures

In [ ]:
# In Julia this struct is immutable
struct HyperParams
    tau             ::Int64
    nOrigins        ::Int64
    nDestinations   ::Int64
    nLanes          ::Int64
    nCarriers       ::Int64
    Bids            ::Vector{Vector{Int64}}
    Winners         ::OrderedDict{Int64, Vector{Int64}}
    CarrierIdx      ::Vector{Vector{Int64}}
    from_           ::Vector{Vector{Int64}}
    to_             ::Vector{Vector{Int64}}
    entry_stock_0   ::Vector{Int64}
    exit_stock_0    ::Vector{Int64}
    exit_short_0    ::Vector{Int64}
    entry_capacity  ::Vector{Int64}
    exit_capacity   ::Vector{Int64}
    flow_support    ::Vector{NTuple}
    entry_store_coef::Vector{Float64}
    exit_store_coef ::Vector{Float64}
    exit_short_coef ::Vector{Float64}
    transport_coef  ::Vector{Float64}
    carrier_capacity::Vector{Int64}
end

# Outer constructor with arguments
function HyperParams(; 
                     tau             ::Int64 = 5,
                     nOrigins        ::Int64 = 2,
                     nDestinations   ::Int64 = 2,
                     nCarriers       ::Int64 = 3,
                     Bids            ::Vector{Vector{Int64}} = [[1, 2, 3, 4],
                                                                [1, 4],
                                                                [2, 3],
                                                                [1, 3, 4],
                                                                [2, 4]],
                     Winners         ::OrderedDict{Int64, Vector{Int64}} = OrderedDict(
                                                                             1 => [2, 3], 
                                                                             2 => [4, 5], 
                                                                             3 => [1]),
                     entry_stock_0   ::Vector{Int64} = [100, 0],
                     exit_stock_0    ::Vector{Int64} = [300, 0],
                     exit_short_0    ::Vector{Int64} = [0, 0],
                     entry_capacity  ::Vector{Int64} = [1000, 1000],
                     exit_capacity   ::Vector{Int64} = [1000, 1000],
                     flow_support    ::Vector{Int64} = [0, 100, 300, 500],
                     entry_store_coef::Vector{Float64} = [2.0, 2.0],
                     exit_store_coef ::Vector{Float64} = [1.0, 1.0],
                     exit_short_coef ::Vector{Float64} = [3.0, 3.0],
                     transport_coef  ::Vector{Float64} = [ 4.8, 3.3, 6, 6.9, 7.3, 5.9, 5.8, 5.4, 3.3, 6.4, 6.9, 5.2, 6.3 ], 
                     carrier_capacity::Vector{Int64} = [160, 100, 140, 120, 100,  # Carrier 1
                                                        140, 100, 160, 180, 100,  # Carrier 2
                                                        300, 300, 300, 300, 300]) # Carrier 3

    # nLanes = nOrigins * nDestinations

    from_ = Dict{Int64, Vector{Int64}}()
    for i in 1:nOrigins
        from_[i] = (collect(1:nDestinations) .- 1) .* nOrigins .+ i
    end

    to_ = Dict{Int64, Vector{Int64}}()
    for j in 1:nDestinations
        to_[j] = (j .- 1) .* nOrigins .+ collect(1:nOrigins)
    end

    ordx = vcat(collect(values(Winners))...)
    Ldx  = vcat(Bids[ordx]...)

    nLc = [0; cumsum([ length(vcat(Bids[Winners[cdx]]...)) for cdx in 1:nCarriers ])]

    HyperParams(
        tau,
        nOrigins,
        nDestinations,
        nLc[end], # nLanes
        nCarriers,
        Bids,
        Winners,
        [ collect(nLc[i] + 1 : nLc[i + 1]) for i in 1:length(nLc) - 1 ],   # CarrierIdx
        [ findall(lane -> lane in from_[i], Ldx) for i in 1:nOrigins ],    # from_
        [ findall(lane -> lane in to_[j], Ldx) for j in 1:nDestinations ], # to_
        entry_stock_0,
        exit_stock_0,
        exit_short_0,
        entry_capacity,
        exit_capacity,
        vec(collect(Base.product(ntuple(_ -> flow_support, nOrigins + nDestinations)...))), # Exhaustive 
        entry_store_coef,
        exit_store_coef,
        exit_short_coef,
        transport_coef,
        carrier_capacity)
end

In [ ]:
function transportation_t(sp::Model, stage::Int64; config::HyperParams)
    ## State variables
    @variables(sp, begin
        0 <= entry[i = 1:config.nOrigins],      (SDDP.State, initial_value = config.entry_stock_0[i])
        0 <= exitp[j = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_stock_0[j])
        0 <= exitm[j = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_short_0[j])
        0 <= move[k  = 1:config.nLanes]
        inflow[ i = 1:config.nOrigins]
        outflow[j = 1:config.nDestinations]
    end)
    ## Decision variables
    ## Constraints
    # Carrier capacity
    @constraint(sp, [k = 1:config.nCarriers], sum(move[config.CarrierIdx[k]]) <= config.carrier_capacity[(k - 1) * config.tau + stage])
    # Entry volume
    @constraint(sp, [i = 1:config.nOrigins], sum(move[config.from_[i]]) <= entry[i].in + inflow[i])
    # Storage limit
    # @constraint(sp, [i = 1:config.nOrigins],      entry[i].in + inflow[i] <= config.entry_capacity[i])
    @constraint(sp, [j = 1:config.nDestinations], exitp[j].in + sum(move[config.to_[j]]) <= config.exit_capacity[j])
    # Transition
    @constraint(sp, [i = 1:config.nOrigins],      entry[i].out == entry[i].in + inflow[i] - sum(move[config.from_[i]]))
    @constraint(sp, [j = 1:config.nDestinations], exitp[j].out - exitm[j].out == exitp[j].in - exitm[j].in + sum(move[config.to_[j]]) - outflow[j])
    # Uncertain variables
    Ξ = config.flow_support
    SDDP.parameterize(sp, Ξ) do ξ
        JuMP.fix.(inflow,  ξ[1:config.nOrigins])
        JuMP.fix.(outflow, ξ[config.nOrigins .+ (1:config.nDestinations)])
    end
    ## Objective function
    @stageobjective(sp, 
        sum(config.entry_store_coef[i] * entry[i].in for i in 1:config.nOrigins)      + 
        sum(config.exit_store_coef[j]  * exitp[j].in for j in 1:config.nDestinations) + 
        sum(config.exit_short_coef[j]  * exitm[j].in for j in 1:config.nDestinations) + 
        # sum(config.transport_coef[(k - 1) * config.tau + stage] * move[l] for k in 1:config.nCarriers for l in config.CarrierIdx[k])
        sum(sum(config.transport_coef[config.CarrierIdx[k]] .* move[config.CarrierIdx[k]]) for k in 1:config.nCarriers)
    )
end

In [ ]:
# Hyperparameter configuration
config = HyperParams(); # Default

In [ ]:
model = SDDP.PolicyGraph(
    (sp, stage) -> transportation_t(sp, stage; config = config),
    SDDP.LinearGraph(config.tau);
    sense       = :Min,
    lower_bound = 0.0,
    optimizer   = Gurobi.Optimizer,
)

In [ ]:
@time SDDP.train(model;
    iteration_limit = 200,
    cut_type        = SDDP.SINGLE_CUT,
    parallel_scheme = SDDP.Serial(),
)

In [ ]:
simulations = SDDP.simulate(
    # The trained model to simulate.
    model,
    # The number of replications.
    10000,
    # A list of names to record the values of.
    [:move, :inflow, :outflow, :entry, :exitp, :exitm],
);

In [ ]:
# obj = map(simulations[1]) do node
#     return node[:stage_objective]
# end

# sum(obj)

# map(simulations[1]) do node
#     return node[:noise_term]
# end

# map(sim -> sum(node[:stage_objective] for node in sim), simulations)

# noise_terms = map(node -> node[:noise_term], simulations[1])
# vcat([collect(ξ)' for ξ in noise_terms]...)

In [ ]:
# Simulated objectives
obj = map(sim -> sum(node[:stage_objective] for node in sim), simulations);

# Scenarios
noise = vcat([ vcat([collect(ξ)' for ξ in map(node -> node[:noise_term], sim)]...) for sim in simulations ]...);

In [ ]:
using HDF5

h5write("/Users/georgios.vassos1/Desktop/5x2x2x3_obj_10e4.h5", "5x2x2x3_obj", obj)
h5write("/Users/georgios.vassos1/Desktop/5x2x2x3_ksi_10e4.h5", "5x2x2x3_ksi", noise)

In [ ]:
using JSON3

# Convert to JSON
json_str = JSON3.pretty(simulations[2][1])

### Scaling the number of supply and demand hubs

In [ ]:
# Hyperparameter configuration
config = HyperParams(
    tau             = 4,
    nOrigins        = 4,
    nDestinations   = 4,
    nCarriers       = 3,
    Bids            = [[1, 2, 3, 4, 5, 6, 7, 8],
                       [5, 6, 7, 8, 9, 10, 11, 12],
                       [10, 11, 12, 13, 14, 15, 16],
                       [1, 2, 3, 4, 8, 9, 10, 11],
                       [5, 6, 7, 8, 13, 14, 15, 16]],
    entry_stock_0   = [500, 500, 500, 500],
    exit_stock_0    = [0, 0, 0, 0],
    exit_short_0    = [0, 0, 0, 0],
    entry_capacity  = [1000, 1000, 1000, 1000],
    exit_capacity   = [1000, 1000, 1000, 1000],
    flow_support    = [0, 100, 300, 500],
    entry_store_coef= [2.0, 2.0, 2.0, 2.0],
    exit_store_coef = [1.0, 1.0, 1.0, 1.0],
    exit_short_coef = [3.0, 3.0, 3.0, 3.0],
    transport_coef  = [ 2.0, 1.7, 2.1, 3.3, 2.0, 1.7, 2.1, 3.3, 2.0, 1.7, 2.1, 3.3, 2.0, 1.7, 2.1, 3.3,   # Carrier 1 / Lane 1 - 16
                        1.7, 1.9, 2.6, 3.7, 1.7, 1.9, 2.6, 3.7, 1.7, 1.9, 2.6, 3.7, 1.7, 1.9, 2.6, 3.7,   # Carrier 2 / Lane 1 - 16
                        2.4, 1.2, 2.9, 3.1, 2.4, 1.2, 2.9, 3.1, 2.4, 1.2, 2.9, 3.1, 2.4, 1.2, 2.9, 3.1 ], # Carrier 3 / Lane 1 - 16
    carrier_capacity= [160, 100, 140, 120,  # Carrier 1 / Time Stage 1 - 4
                       140, 100, 160, 180,  # Carrier 2 / Time Stage 1 - 4
                       300, 300, 300, 300 ] # Carrier 3 / Time Stage 1 - 4
);

In [ ]:
model = SDDP.PolicyGraph(
    (sp, stage) -> transportation_t(sp, stage; config = config),
    SDDP.LinearGraph(config.tau);
    sense       = :Min,
    lower_bound = 0.0,
    optimizer   = Gurobi.Optimizer,
)

In [ ]:
SDDP.train(model;
    iteration_limit = 200,
    cut_type        = SDDP.SINGLE_CUT,
    parallel_scheme = SDDP.Serial(),
)

### Scaling the number of carriers

In [ ]:
using Distributions
# # Sample n numbers
# samples = map(x -> round(x, 2), rand(Uniform(4.0, 8.0), sum(map(length, config.CarrierIdx))))

# Compute the total length of all elements in config.CarrierIdx
n = sum(map(length, config.CarrierIdx))

# Sample n numbers and round to 2 decimal places
coef = map(x -> round(x, digits = 2), rand(Uniform(4.0, 8.0), n))

println(join(coef, ", "))

In [ ]:
# Hyperparameter configuration
config = HyperParams(
    tau              = 5,
    nOrigins         = 2,
    nDestinations    = 2,
    nCarriers        = 9,
    Bids             = [[1, 2, 3, 4],
                        [1, 4],
                        [2, 3],
                        [1, 3, 4],
                        [2, 4]],
    Winners          = OrderedDict(
                         1 => [1, 2], 
                         2 => [3], 
                         3 => [1], 
                         4 => [1, 3], 
                         5 => [2, 4], 
                         6 => [5], 
                         7 => [3, 5], 
                         8 => [4], 
                         9 => [2, 4]
                       ),
    entry_stock_0    = [0, 0],
    exit_stock_0     = [22, 0],
    exit_short_0     = [0, 0],
    entry_capacity   = [5000, 5000],
    exit_capacity    = [5000, 5000],
    flow_support     = [0, 100, 300, 500],
    entry_store_coef = [2.0, 2.0],
    exit_store_coef  = [1.0, 1.0],
    exit_short_coef  = [3.0, 3.0],
    transport_coef   = [ 7.04, 7.19, 4.03, 6.66, 4.87, 5.61, 
                         5.42, 5.83, 
                         4.01, 4.18, 5.79, 6.24, 
                         7.46, 5.27, 6.15, 5.95, 5.15, 4.09, 
                         4.44, 7.33, 4.22, 7.84, 7.3, 
                         4.26, 6.8, 
                         4.4, 4.02, 7.34, 7.86, 
                         5.58, 7.34, 5.82, 
                         5.15, 6.2, 4.28, 7.23, 6.33 ], 
    carrier_capacity = [70, 40, 40, 80, 30,  # Carrier 1
                        90, 40, 60, 70, 30,  # Carrier 2
                        70, 50, 80, 60, 30,  # Carrier 3
                        30, 60, 30, 90, 60,  # Carrier 4
                        60, 70, 50, 50, 60,  # Carrier 5
                        60, 80, 50, 40, 60,  # Carrier 6
                        30, 60, 80, 40, 90,  # Carrier 7
                        70, 70, 80, 40, 90,  # Carrier 8
                        60, 30, 60, 90, 90]  # Carrier 9
);

In [ ]:
model = SDDP.PolicyGraph(
    (sp, stage) -> transportation_t(sp, stage; config = config),
    SDDP.LinearGraph(config.tau);
    sense       = :Min,
    lower_bound = 0.0,
    optimizer   = Gurobi.Optimizer,
)

In [ ]:
SDDP.train(model;
    iteration_limit = 200,
    cut_type        = SDDP.SINGLE_CUT,
    parallel_scheme = SDDP.Serial(),
)

### Scaling the length of the planning horizon

In [ ]:
# Hyperparameter configuration
config = HyperParams(
    tau              = 56,
    nOrigins         = 2,
    nDestinations    = 2,
    nCarriers        = 9,
    Bids             = [[1, 2, 3, 4],
                        [1, 4],
                        [2, 3],
                        [1, 3, 4],
                        [2, 4]],
    Winners          = OrderedDict(
                         1 => [1, 2], 
                         2 => [3], 
                         3 => [1], 
                         4 => [1, 3], 
                         5 => [2, 4], 
                         6 => [5], 
                         7 => [3, 5], 
                         8 => [4], 
                         9 => [2, 4]
                       ),
    entry_stock_0    = [0, 0],
    exit_stock_0     = [22, 0],
    exit_short_0     = [0, 0],
    entry_capacity   = [5000, 5000],
    exit_capacity    = [5000, 5000],
    flow_support     = [0, 100, 300, 500],
    entry_store_coef = [2.0, 2.0],
    exit_store_coef  = [1.0, 1.0],
    exit_short_coef  = [3.0, 3.0],
    transport_coef   = [ 7.04, 7.19, 4.03, 6.66, 4.87, 5.61, 
                         5.42, 5.83, 
                         4.01, 4.18, 5.79, 6.24, 
                         7.46, 5.27, 6.15, 5.95, 5.15, 4.09, 
                         4.44, 7.33, 4.22, 7.84, 7.3, 
                         4.26, 6.8, 
                         4.4, 4.02, 7.34, 7.86, 
                         5.58, 7.34, 5.82, 
                         5.15, 6.2, 4.28, 7.23, 6.33 ], 
    carrier_capacity = rand(30:10:90, 56 * 9)
    # carrier_capacity = [70, 40, 40, 80, 30, 90, 40, 60, 70, 30, 50, 50, # Carrier 1
    #                     90, 40, 60, 70, 30, 60, 80, 50, 40, 60, 50, 60, # Carrier 2
    #                     70, 50, 80, 60, 30, 60, 70, 50, 50, 60, 70, 80, # Carrier 3
    #                     30, 60, 30, 90, 60, 70, 70, 80, 40, 90, 30, 90, # Carrier 4
    #                     60, 70, 50, 50, 60, 70, 70, 80, 40, 50, 40, 40, # Carrier 5
    #                     60, 80, 50, 40, 60, 60, 80, 50, 40, 60, 60, 80, # Carrier 6
    #                     30, 60, 80, 40, 90, 60, 80, 60, 50, 60, 80, 50, # Carrier 7
    #                     70, 70, 80, 40, 90, 60, 60, 50, 40, 90, 90, 90, # Carrier 8
    #                     60, 30, 60, 90, 90, 60, 80, 50, 60, 60, 40, 60 ]# Carrier 9
);

In [ ]:
model = SDDP.PolicyGraph(
    (sp, stage) -> transportation_t(sp, stage; config = config),
    SDDP.LinearGraph(config.tau);
    sense       = :Min,
    lower_bound = 0.0,
    optimizer   = Gurobi.Optimizer,
)

In [ ]:
SDDP.train(model;
    iteration_limit = 200,
    cut_type        = SDDP.SINGLE_CUT,
    parallel_scheme = SDDP.Serial(),
)

## Parallelization of the Multi-Dimensional Instance

In [ ]:
using Distributed
Distributed.addprocs(10);
@everywhere using JuMP, SDDP, Gurobi, DataStructures

In [ ]:
# In Julia this struct is immutable
@everywhere struct HyperParams
    tau             ::Int64
    nOrigins        ::Int64
    nDestinations   ::Int64
    nLanes          ::Int64
    nCarriers       ::Int64
    Bids            ::Vector{Vector{Int64}}
    Winners         ::OrderedDict{Int64, Vector{Int64}}
    CarrierIdx      ::Vector{Vector{Int64}}
    from_           ::Vector{Vector{Int64}}
    to_             ::Vector{Vector{Int64}}
    entry_stock_0   ::Vector{Int64}
    exit_stock_0    ::Vector{Int64}
    exit_short_0    ::Vector{Int64}
    entry_capacity  ::Vector{Int64}
    exit_capacity   ::Vector{Int64}
    flow_support    ::Vector{NTuple}
    entry_store_coef::Vector{Float64}
    exit_store_coef ::Vector{Float64}
    exit_short_coef ::Vector{Float64}
    transport_coef  ::Vector{Float64}
    carrier_capacity::Vector{Int64}
end

# Outer constructor with arguments
@everywhere function HyperParams(; 
                     tau             ::Int64 = 4,
                     nOrigins        ::Int64 = 2,
                     nDestinations   ::Int64 = 2,
                     nCarriers       ::Int64 = 3,
                     Bids            ::Vector{Vector{Int64}} = [[1, 2, 3, 4],
                                                                [1, 3],
                                                                [2, 4],
                                                                [1, 2],
                                                                [3, 4]],
                     Winners         ::OrderedDict{Int64, Vector{Int64}} = OrderedDict(
                                                                             1 => [2, 3], 
                                                                             2 => [4, 5], 
                                                                             3 => [1]),
                     entry_stock_0   ::Vector{Int64} = [5, 5],
                     exit_stock_0    ::Vector{Int64} = [0, 0],
                     exit_short_0    ::Vector{Int64} = [0, 0],
                     entry_capacity  ::Vector{Int64} = [50, 50],
                     exit_capacity   ::Vector{Int64} = [50, 50],
                     flow_support    ::Vector{Int64} = collect(1:10),
                     entry_store_coef::Vector{Float64} = [2.0, 2.0],
                     exit_store_coef ::Vector{Float64} = [1.0, 1.0],
                     exit_short_coef ::Vector{Float64} = [3.0, 3.0],
                     transport_coef  ::Vector{Float64} = [ 2.0, 1.7, 2.1, 3.3,   # Carrier 1
                                                           1.7, 1.9, 2.6, 3.7,   # Carrier 2
                                                           2.4, 1.2, 2.9, 3.1 ], # Carrier 3
                     carrier_capacity::Vector{Int64} = [ 8,  5,  7, 20, 
                                                         7,  5,  8, 20, 
                                                        12,  8,  8, 11])

    # nLanes = nOrigins * nDestinations

    from_ = Dict{Int64, Vector{Int64}}()
    for i in 1:nOrigins
        from_[i] = (collect(1:nDestinations) .- 1) .* nOrigins .+ i
    end

    to_ = Dict{Int64, Vector{Int64}}()
    for j in 1:nDestinations
        to_[j] = (j .- 1) .* nOrigins .+ collect(1:nOrigins)
    end

    ordx = vcat(collect(values(Winners))...)
    Ldx  = vcat(Bids[ordx]...)

    nLc = [0; cumsum([ length(vcat(Bids[Winners[cdx]]...)) for cdx in 1:nCarriers ])]

    HyperParams(
        tau,
        nOrigins,
        nDestinations,
        nLc[end], # nLanes
        nCarriers,
        Bids,
        Winners,
        [ collect(nLc[i] + 1 : nLc[i + 1]) for i in 1:length(nLc) - 1 ], # CarrierIdx
        [ findall(lane -> lane in from_[i], Ldx) for i in 1:nOrigins ],
        [ findall(lane -> lane in to_[j], Ldx) for j in 1:nDestinations ],
        entry_stock_0,
        exit_stock_0,
        exit_short_0,
        entry_capacity,
        exit_capacity,
        vec(collect(Base.product(ntuple(_ -> flow_support, nOrigins + nDestinations)...))),
        entry_store_coef,
        exit_store_coef,
        exit_short_coef,
        transport_coef,
        carrier_capacity)
end

In [ ]:
@everywhere function transportation_t(sp::Model, stage::Int64; config::HyperParams)
    ## State variables
    @variables(sp, begin
        0 <= entry[i = 1:config.nOrigins],      (SDDP.State, initial_value = config.entry_stock_0[i])
        0 <= exitp[j = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_stock_0[j])
        0 <= exitm[j = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_short_0[j])
        0 <= move[k  = 1:config.nLanes]
        inflow[ i = 1:config.nOrigins]
        outflow[j = 1:config.nDestinations]
    end)
    ## Decision variables
    ## Constraints
    # Carrier capacity
    @constraint(sp, [k = 1:config.nCarriers], sum(move[config.CarrierIdx[k]]) <= config.carrier_capacity[(k - 1) * config.tau + stage])
    # # Entry volume
    # @constraint(sp, [i = 1:config.nOrigins], sum(move[config.from_[i]]) <= entry[i].in + inflow[i])
    # Storage limit
    @constraint(sp, [i = 1:config.nOrigins],      entry[i].in + inflow[i] <= config.entry_capacity[i])
    @constraint(sp, [j = 1:config.nDestinations], exitp[j].in + sum(move[config.to_[j]]) <= config.exit_capacity[j])
    # Transition
    @constraint(sp, [i = 1:config.nOrigins],      entry[i].out == entry[i].in + inflow[i] - sum(move[config.from_[i]]))
    @constraint(sp, [j = 1:config.nDestinations], exitp[j].out - exitm[j].out == exitp[j].in - exitm[j].in + sum(move[config.to_[j]]) - outflow[j])
    # Uncertain variables
    Ξ = config.flow_support
    SDDP.parameterize(sp, Ξ) do ξ
        JuMP.fix.(inflow,  ξ[1:config.nOrigins])
        JuMP.fix.(outflow, ξ[config.nOrigins .+ (1:config.nDestinations)])
    end
    ## Objective function
    @stageobjective(sp, 
        sum(config.entry_store_coef[i] * entry[i].in for i in 1:config.nOrigins)      + 
        sum(config.exit_store_coef[j]  * exitp[j].in for j in 1:config.nDestinations) + 
        sum(config.exit_short_coef[j]  * exitm[j].in for j in 1:config.nDestinations) + 
        # sum(config.transport_coef[(k - 1) * config.tau + stage] * move[l] for k in 1:config.nCarriers for l in config.CarrierIdx[k])
        sum(sum(config.transport_coef[(k - 1) * config.tau + stage] * move[config.CarrierIdx[k]]) for k in 1:config.nCarriers)
    )
end

In [ ]:
# Hyperparameter configuration
@everywhere config = HyperParams(); # Default

In [ ]:
model = SDDP.PolicyGraph(
    (sp, stage) -> transportation_t(sp, stage; config = config),
    SDDP.LinearGraph(config.tau);
    sense       = :Min,
    lower_bound = 0.0,
    optimizer   = Gurobi.Optimizer,
)

In [ ]:
SDDP.train(
    model;
    iteration_limit = 10,
    cut_type        = SDDP.SINGLE_CUT,
    parallel_scheme = SDDP.Asynchronous() do m::SDDP.PolicyGraph
        env = Gurobi.Env()
        set_optimizer(m, () -> Gurobi.Optimizer(env))
    end,
)

In [ ]:
# function transportation_t(sp::Model, stage::Int64; config::HyperParams)
#     ## State variables
#     @variables(sp, begin
#         0 <= entry, (SDDP.State, initial_value = config.entry_stock_0[1])
#         0 <= exitp, (SDDP.State, initial_value = config.exit_stock_0[1])
#         0 <= exitm, (SDDP.State, initial_value = config.exit_short_0[1])
#         0 <= move[l = 1:config.nLanes]
#         inflow
#         outflow
#     end)
#     ## Decision variables
#     ## Constraints
#     # Carrier capacity
#     @constraint(sp, sum(move) <= entry.in + inflow)
#     # Storage limit
#     @constraint(sp, entry.in + inflow    <= config.entry_capacity[1])
#     @constraint(sp, exitp.in + sum(move) <= config.exit_capacity[1])
#     # Transition
#     @constraint(sp, entry.out == entry.in + inflow - sum(move))
#     @constraint(sp, exitp.out - exitm.out == exitp.in - exitm.in + sum(move) - outflow)
#     # Uncertain variables
#     Ξ = vec(collect(Base.product(ntuple(_ -> config.flow_support, config.nOrigins + config.nDestinations)...)))
#     SDDP.parameterize(sp, Ξ) do ξ
#         JuMP.fix.(inflow,  ξ[1])
#         JuMP.fix.(outflow, ξ[2])
#     end
#     ## Objective function
#     @stageobjective(sp, 
#         config.entry_store_coef[1] * entry.in + 
#         config.exit_store_coef[1]  * exitp.in + 
#         config.exit_short_coef[1]  * exitm.in + 
#         sum(config.transport_coef[l] * move[l] for l in 1:config.nLanes)
#     )
# end

In [ ]:
# # Hyperparameter configuration
# config = HyperParams(tau             = 4,
#                      nOrigins        = 1,
#                      nDestinations   = 1,
#                      nCarriers       = 2,
#                      Bids            = [[1],
#                                         [1]],
#                      Winners         = OrderedDict(
#                                          1 => [1], 
#                                          2 => [2]),
#                      entry_stock_0   = [5],
#                      exit_stock_0    = [0],
#                      exit_short_0    = [0],
#                      entry_capacity  = [50],
#                      exit_capacity   = [50],
#                      flow_support    = collect(1:10),
#                      entry_store_coef= [2.0],
#                      exit_store_coef = [1.0],
#                      exit_short_coef = [3.0],
#                      transport_coef  = [ 1.1, 0.7 ])

## Realistic Uncertainty Model

### Pointwise simulation of interval count series

In [ ]:
using Distributions, LinearAlgebra, Random, Plots

In [ ]:
function rpoisar1(t, p, lambda_t, params)
    # Extract parameters
    d = params[:linear][:d]
    A = params[:linear][:A]
    B = params[:linear][:B]
    tau = params[:seasonal][:T]
    ampl = params[:seasonal][:A]
    R = params[:corrmat]
    K = params[:K]

    # Generate correlated uniform random variables
    mvn = MvNormal(R)
    z = rand(mvn, K)  # Generates a matrix of size (p, K)
    u = cdf.(Normal(0, 1), z)  # Elementwise CDF transformation (size: p × K)

    # Transform to exponential variables
    x = -log.(u) ./ reshape(lambda_t, p, 1)  # Reshape lambda_t for broadcasting

    # Calculate cumulative sums and counts
    y_t = [sum(cumsum(x[j, :]) .<= 1.0) for j in 1:p]

    # Update lambda for the next step
    lambda_t = d .+ (A * lambda_t) .+ (B * y_t) .+ ampl * cos.(2.0 * π * ((t + 1) / tau))

    # Return results for the single time step
    return Dict("y_t" => y_t, "lambda_t" => lambda_t)
end

In [ ]:
# Define the parameters as in the R example
uncertainty_params = Dict(
    :linear => Dict(
        :d => [0.5, 0.4, 0.5, 0.8],
        :A => Diagonal([0.7, 0.8, 0.6, 0.5]),
        :B => Diagonal([0.2, 0.1, 0.3, 0.3])
    ),
    :seasonal => Dict(
        :T => 365.0,
        :A => 0.2
    ),
    :corrmat => [
        1.0 0.3 0.5 0.7;
        0.3 1.0 0.4 0.2;
        0.5 0.4 1.0 0.6;
        0.7 0.2 0.6 1.0
    ],
    :K => 20
)

N = 365
p = 4

# Initialize arrays for results
Y = zeros(Int, N, p)
lambda_t = [5.0, 3.0, 9.0, 4.0]
# Simulate for N time points
for t in 1:N
    result   = rpoisar1(t, p, lambda_t, uncertainty_params)
    Y[t, :] .= result["y_t"]
    lambda_t = result["lambda_t"]
end

In [ ]:
# Select a backend suitable for JupyterLab (GR works well in most cases)
gr()

# Create a plot object using steppost for step-like visualization
plt = plot(
    1:N, Y[:, 1],
    label="Flow 1",
    seriestype=:steppost,
    xlabel="Time",
    ylabel="Count",
    title="Simulated Poisson AR(1) Processes",
    size=(1200,800)
)

# Add remaining series to the plot
for i in 2:p
    plot!(
        plt,
        1:N, Y[:, i],
        label="Flow $i",
        seriestype=:steppost
    )
end

# Explicitly display the plot in JupyterLab
display(plt)

### Optimization under the realistic uncertainty model

In [ ]:
using JuMP, SDDP
using HiGHS, Gurobi
using DataStructures
using LinearAlgebra
using Distributions
using Statistics, Test

In [ ]:
# In Julia this struct is immutable
struct HyperParams
    tau             ::Int64
    nOrigins        ::Int64
    nDestinations   ::Int64
    nLanes          ::Int64
    nCarriers       ::Int64
    nODs            ::Int64
    Bids            ::Vector{Vector{Int64}}
    Winners         ::OrderedDict{Int64, Vector{Int64}}
    CarrierIdx      ::Vector{Vector{Int64}}
    from_           ::Vector{Vector{Int64}}
    to_             ::Vector{Vector{Int64}}
    entry_stock_0   ::Vector{Int64}
    exit_stock_0    ::Vector{Int64}
    exit_short_0    ::Vector{Int64}
    entry_capacity  ::Vector{Int64}
    exit_capacity   ::Vector{Int64}
    flow_support    ::Vector{NTuple}
    entry_store_coef::Vector{Float64}
    exit_store_coef ::Vector{Float64}
    exit_short_coef ::Vector{Float64}
    transport_coef  ::Vector{Float64}
    carrier_capacity::Vector{Int64}
    lambda_0        ::Vector{Float64}
    u_params        ::Dict{Symbol, Any}
end

# Outer constructor with arguments
function HyperParams(; 
                     tau             ::Int64 = 4,
                     nOrigins        ::Int64 = 2,
                     nDestinations   ::Int64 = 2,
                     nCarriers       ::Int64 = 3,
                     Bids            ::Vector{Vector{Int64}} = [[1, 2, 3, 4],
                                                                [1, 3],
                                                                [2, 4],
                                                                [1, 2],
                                                                [3, 4]],
                     Winners         ::OrderedDict{Int64, Vector{Int64}} = OrderedDict(
                                                                             1 => [2, 3], 
                                                                             2 => [4, 5], 
                                                                             3 => [1]),
                     entry_stock_0   ::Vector{Int64} = [5, 5],
                     exit_stock_0    ::Vector{Int64} = [0, 0],
                     exit_short_0    ::Vector{Int64} = [0, 0],
                     entry_capacity  ::Vector{Int64} = [50, 50],
                     exit_capacity   ::Vector{Int64} = [50, 50],
                     flow_support    ::Vector{Int64} = [0, 5, 10],
                     entry_store_coef::Vector{Float64} = [2.0, 2.0],
                     exit_store_coef ::Vector{Float64} = [1.0, 1.0],
                     exit_short_coef ::Vector{Float64} = [3.0, 3.0],
                     transport_coef  ::Vector{Float64} = [ 2.0, 1.7, 2.1, 3.3,   # Carrier 1 / Lanes 1 - 4
                                                           1.7, 1.9, 2.6, 3.7,   # Carrier 2 / Lanes 1 - 4
                                                           2.4, 1.2, 2.9, 3.1 ], # Carrier 3 / Lanes 1 - 4
                     carrier_capacity::Vector{Int64} = [ 8,  5,  7, 20,  # Carrier 1 / Time Stage 1 - 4
                                                         7,  5,  8, 20,  # Carrier 2 / Time Stage 1 - 4
                                                        12,  8,  8, 11], # Carrier 3 / Time Stage 1 - 4
                     lambda_0::Vector{Float64} = [5.0, 3.0, 9.0, 4.0],
                     u_params::Dict{Symbol, Any} = Dict(
                                            :linear => Dict(
                                                :d => [0.5, 0.4, 0.5, 0.8],
                                                :A => Diagonal([0.7, 0.8, 0.6, 0.5]),
                                                :B => Diagonal([0.2, 0.1, 0.3, 0.3])
                                            ),
                                            :seasonal => Dict(
                                                :T => 365.0,
                                                :A => 0.2
                                            ),
                                            :corrmat => [
                                                1.0 0.3 0.5 0.7;
                                                0.3 1.0 0.4 0.2;
                                                0.5 0.4 1.0 0.6;
                                                0.7 0.2 0.6 1.0
                                            ],
                                            :K => 20
                                        ))

    # nLanes = nOrigins * nDestinations

    from_ = Dict{Int64, Vector{Int64}}()
    for i in 1:nOrigins
        from_[i] = (collect(1:nDestinations) .- 1) .* nOrigins .+ i
    end

    to_ = Dict{Int64, Vector{Int64}}()
    for j in 1:nDestinations
        to_[j] = (j .- 1) .* nOrigins .+ collect(1:nOrigins)
    end

    ordx = vcat(collect(values(Winners))...)
    Ldx  = vcat(Bids[ordx]...)

    nLc = [0; cumsum([ length(vcat(Bids[Winners[cdx]]...)) for cdx in 1:nCarriers ])]

    HyperParams(
        tau,
        nOrigins,
        nDestinations,
        nLc[end], # nLanes
        nCarriers,
        nOrigins + nDestinations,
        Bids,
        Winners,
        [ collect(nLc[i] + 1 : nLc[i + 1]) for i in 1:length(nLc) - 1 ], # CarrierIdx
        [ findall(lane -> lane in from_[i], Ldx) for i in 1:nOrigins ],
        [ findall(lane -> lane in to_[j], Ldx) for j in 1:nDestinations ],
        entry_stock_0,
        exit_stock_0,
        exit_short_0,
        entry_capacity,
        exit_capacity,
        vec(collect(Base.product(ntuple(_ -> flow_support, nOrigins + nDestinations)...))),
        entry_store_coef,
        exit_store_coef,
        exit_short_coef,
        transport_coef,
        carrier_capacity,
        lambda_0,
        u_params)
end

mutable struct Auxilia
    lambda::Matrix{Float64}
end

In [ ]:
function rmunif(R::Matrix{Float64}, K::Int64)
    # Generate correlated uniform random variables
    mvn = MvNormal(R)
    z = rand(mvn, K)  # Generates a matrix of size (p, K)
    u = cdf.(Normal(0, 1), z)  # Elementwise CDF transformation (size: p × K)

    return u
end

function transportation_t(sp::Model, stage::Int64; config::HyperParams, external::Auxilia)
    ## State variables
    @variables(sp, begin
        0 <= entry[i  = 1:config.nOrigins],      (SDDP.State, initial_value = config.entry_stock_0[i])
        0 <= exitp[j  = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_stock_0[j])
        0 <= exitm[j  = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_short_0[j])
        0 <= move[k   = 1:config.nLanes]
        inflow[ i = 1:config.nOrigins]
        outflow[j = 1:config.nDestinations]
    end)
    ## Constraints
    # Carrier capacity
    @constraint(sp, [k = 1:config.nCarriers], sum(move[config.CarrierIdx[k]]) <= config.carrier_capacity[(k - 1) * config.tau + stage])
    # Entry volume
    @constraint(sp, [i = 1:config.nOrigins], sum(move[config.from_[i]]) <= entry[i].in + inflow[i])
    # Storage limit
    # @constraint(sp, [i = 1:config.nOrigins],      entry[i].in + inflow[i] <= config.entry_capacity[i])
    @constraint(sp, [j = 1:config.nDestinations], exitp[j].in + sum(move[config.to_[j]]) <= config.exit_capacity[j])
    # Transition
    @constraint(sp, [i = 1:config.nOrigins],      entry[i].out == entry[i].in + inflow[i] - sum(move[config.from_[i]]))
    @constraint(sp, [j = 1:config.nDestinations], exitp[j].out - exitm[j].out == exitp[j].in - exitm[j].in + sum(move[config.to_[j]]) - outflow[j])
    # Uncertain variables
    U = [(idx, rmunif(config.u_params[:corrmat], config.u_params[:K])) for idx in 1:100]
    SDDP.parameterize(sp, U) do u_enumerated
        idx = u_enumerated[1]
        u   = u_enumerated[2]
        # Transform to exponential variables
        x = -log.(u) ./ reshape(external.lambda[idx,:], config.nODs, 1)  # Reshape lambda_t for broadcasting
        # Calculate cumulative sums and counts
        ξ = [sum(cumsum(x[j, :]) .<= 1.0) for j in 1:config.nODs]

        JuMP.fix.(inflow,  ξ[1:config.nOrigins])
        JuMP.fix.(outflow, ξ[config.nOrigins .+ (1:config.nDestinations)])

        external.lambda[idx,:] = config.u_params[:linear][:d] .+ 
            (config.u_params[:linear][:A] * external.lambda[idx,:]) .+ 
            (config.u_params[:linear][:B] * ξ) .+ 
             config.u_params[:seasonal][:A] * cos.(2.0 * π * ((stage + 1) / config.u_params[:seasonal][:T]))
    end
    ## Objective function
    @stageobjective(sp, 
        sum(config.entry_store_coef[i] * entry[i].in for i in 1:config.nOrigins)      + 
        sum(config.exit_store_coef[j]  * exitp[j].in for j in 1:config.nDestinations) + 
        sum(config.exit_short_coef[j]  * exitm[j].in for j in 1:config.nDestinations) + 
        sum(sum(config.transport_coef[(k - 1) * config.tau + stage] * move[config.CarrierIdx[k]]) for k in 1:config.nCarriers)
    )
end

In [ ]:
# Hyperparameter configuration
config = HyperParams(); # Default

In [ ]:
model = SDDP.PolicyGraph(
    (sp, stage) -> transportation_t(sp, stage; config = config, external = Auxilia(repeat(config.lambda_0', 100, 1))),
    SDDP.LinearGraph(config.tau);
    sense       = :Min,
    lower_bound = 0.0,
    optimizer   = HiGHS.Optimizer,
)

In [ ]:
SDDP.train(model;
    iteration_limit = 200,
    cut_type        = SDDP.SINGLE_CUT,
    parallel_scheme = SDDP.Serial(),
)

# Recovering `i` and `j` from a 1-Indexed Linear Index in Julia

To recover the row and column indices \( i \) and \( j \) from a 1-indexed linear index \( \ell \) when given \( \ell = (i - 1) \times m + j \) in Julia, you can use the following formulas:

1. Compute $i$ by dividing and flooring: $ i = \left\lfloor \dfrac{\ell - 1}{m} \right\rfloor + 1 $

2. Compute $j$ using the remainder: $ j = (\ell - 1) \bmod m + 1 $


In [ ]:
function get_2d_index(ell, m)
    i = div(ell - 1, m) + 1
    j = (ell - 1) % m + 1
    return i, j
end

In [ ]:
m = 4
i, j = get_2d_index(13, m)
println("i: $i, j: $j")

In [ ]:
using Random

n = 2  # Number of rows
m = 2  # Number of columns
K = 2  # Number of suppliers

# Creating a dictionary to store the 1D indices for each k
kdx = Dict{Int, Vector{Int}}()

# Initialize the dictionary with empty arrays for each k
for k in 1:K
    kdx[k] = Int[]
end

# Loop through each (i, j) and assign indices to suppliers
for i in 1:n
    for j in 1:m
        # Convert (i, j) to a 1D index
        ell = (i - 1) * m + j

        for k in 1:K
            if rand() > 0.5 || ((n <= 2) && (m <= 2))
                # Add the 1D index to the corresponding supplier's list in the dictionary
                push!(kdx[k], ell)
            end
        end
    end
end

# Print the dictionary
for (k, indices) in kdx
    println("Supplier ", k, ": ", indices)
end

In [ ]:
n = 3
m = 2

# Generate 1D indices for row i
i = 1
idx = (collect(1:m) .- 1) .* n .+ i
println("Indices for row \$i: ", idx)

# Generate 1D indices for column j
j = 2
jdx = (j .- 1) .* n .+ collect(1:n)
println("Indices for column \$j: ", jdx)

In [ ]:
intersect(kdx[2], idx)

In [ ]:
# function transportation_t(sp::Model, stage::Int64; n::Int64, m::Int64)
#     @variable(sp, x[i = 1:(n + m)],
#         base_name   = "x",
#         lower_bound = i <= n ? 0 : -10,
#         upper_bound = 10,
#         SDDP.State,
#         initial_value = 0
#     )
# end